# First analysis osmFISH

Importing libraries

In [ ]:
from starfish import Experiment
from starfish import display
import numpy as np
import napari

In [ ]:
from starfish.image import ApplyTransform, LearnTransform, Filter, Segment
from starfish.types import Axes
from starfish.spots import DecodeSpots, FindSpots, AssignTargets

Load data: primary images, nuclei, dots and codebook.

In [ ]:
path = r'Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/output/' 
experiment = Experiment.from_json(path + "experiment.json")

In [ ]:
fov = experiment['fov_000']
imgs = fov.get_image("primary")
nuclei = fov.get_image("nuclei")
#dots = fov.get_image("dots")

In [ ]:
codebook = experiment.codebook

Visualise imported data

In [ ]:
%gui qt
viewer = display(imgs)
viewer.layers[0].name = "raw stack" # rename the layer

In [ ]:
viewer = display(stack=nuclei, viewer=viewer)

In [ ]:
viewer = display(stack=imgs, spots=decoded, masks=masks, viewer=viewer)

In [ ]:
viewer = display(imgs)
viewer.add_image(dots.xarray, name='dots')
viewer.add_labels(spots.to_label_image().label_image, name='cells')

In [ ]:
%gui qt
viewer = display(imgs)
viewer = display(imgs_wth, viewer=viewer)

# Filtering

Filter with white top hat or clipping to remove autofluorescence.

In [ ]:
def filter_white_tophat(imgs, masking_radius):
    wth = Filter.WhiteTophat(masking_radius=masking_radius)
    return wth.run(imgs)

In [ ]:
# filter
#imgs_wth = filter_white_tophat(imgs, 50)

In [ ]:
def clip(image): 
    clip_97 = Filter.Clip(p_min=99)
    return clip_97.run(image)

In [ ]:
imgs_wth = clip(imgs)

# Registration

In [ ]:
def register(imgs, dots, method = 'translation'):
    mip_imgs = imgs.reduce(dims = [Axes.CH, Axes.ZPLANE], func="max")
    mip_dots = dots.reduce(dims = [Axes.CH, Axes.ZPLANE], func="max")
    learn_translation = LearnTransform.Translation(reference_stack=mip_dots, axes=Axes.ROUND, upsampling=1000)
    transforms_list = learn_translation.run(mip_imgs)
    warp = ApplyTransform.Warp()
    registered_imgs = warp.run(imgs, transforms_list=transforms_list, in_place=False, verbose=True)
    return registered_imgs

In [ ]:
# register
registered_imgs = register(imgs_wth, dots_wth)

In [ ]:
projected_imgs = imgs_wth.reduce({Axes.CH}, func="max")

# Finding spots

In [ ]:
tlmpf = FindSpots.TrackpyLocalMaxPeakFinder(
    spot_diameter=5,
    min_mass=0.2,
    max_size=2,
    separation=7,
    noise_size=0.65,
    preprocess=False,
    percentile=10,
    verbose=True,
    is_volume=True,
)
spots = tlmpf.run(image_stack=imgs_wth)

Visualisation of spots tracked

In [ ]:
import trackpy
from tifffile import imread
import matplotlib.pyplot as plt

In [ ]:
f = trackpy.locate(np.concatenate((raw_image,raw_image_1), axis =0), diameter=5, minmass=0.2)

In [ ]:
trackpy.annotate(f, raw_image)

# Decoding spot

In [ ]:
def decode_spots(codebook, spots):
    #decoder = DecodeSpots.PerRoundMaxChannel(codebook=codebook)
    decoder = DecodeSpots.SimpleLookupDecoder(codebook=codebook)
    return decoder.run(spots=spots)

In [ ]:
# decode
decoded = decode_spots(codebook, spots)

# Segmenting cells

In [ ]:
def segment(imgs, nuclei):
    # set parameters
    dapi_thresh = .1  # global threshold value for nuclei images
    stain_thresh = .42  # global threshold value for primary images
    min_dist = 57  # minimum distance (pixels) between nuclei distance transformed peaks

    seg = Segment.Watershed(
        nuclei_threshold=dapi_thresh,
        input_threshold=stain_thresh,
        min_distance=min_dist
    )

    # masks is BinaryMaskCollection for downstream steps
    masks = seg.run(imgs_wth, nuclei)
    return seg, masks

In [ ]:
# segment
seg, masks = segment(imgs, nuclei)
# display intermediate images and result
seg.show()

# Single cell gene expression

In [ ]:
def make_expression_matrix(masks, decoded):
    al = AssignTargets.Label()
    print(al)
    labeled = al.run(masks, decoded[decoded.target != 'nan'])
    cg = labeled[labeled.cell_id != 'nan'].to_expression_matrix()
    return cg

In [ ]:
# make expression matrix
mat = make_expression_matrix(masks, decoded)

In [ ]:
AssignTargets?

# Visualisation results

In [ ]:
print(decoded.to_features_dataframe().head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(30, 10))
sns.set(font_scale=4)
sns.heatmap(mat.data.T,
            yticklabels=['gene 3GAPDH','gene 5GAPDH'],
            xticklabels = ['cell {}'.format(n+1) for n in range(mat.data.T.size)],
            cmap='magma')